<a href="https://colab.research.google.com/github/oneryigit/text_analysis/blob/main/berttopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topic Modeling Using `BerTopic`**


In [ ]:
pip install bertopic

In [ ]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/data/stm_hutbe_01.csv')

In [ ]:
def chunks(s, c):
    return [s[i:i + c] for i in range(0, len(s), c)]

df['content'] = df['content'].apply(lambda x: chunks(x.split(),80))
df = df.explode('content').reset_index(drop=True)
df['content'] = df['content'].apply(' '.join, 1)

In [ ]:
df['content'][1]

'doğrusu yapıp ettiklerinden kalpleri pas tutmuştur anlatılan pas budur bugün yürek paslanmasını kalp kararmasını yaşamaktayız suyun topraktan çekilmesi merhamet insanlığın vicdanından hızla çekiliyor merhametin açtığı boşluğu şiddet öfke zorbalık doldurmakta ahlaki yozlaşmalar vicdani duygulardan yoksunluk hayatın alanında gün geçtikçe artıyor insanlık hengâmede oluş yaratılış hikmetinden uzaklaşıyor şiddet maalesef kültür inanç ırk coğrafya tanımaksızın dünyayı kasıp kavuruyor hemen günümüzü zulüm cinayet haksızlıklara şahit olmanın vicdanımızda açtığı yaralar yüreğimizde bıraktığı sızılarla yaşamaktayız rabbimizin mesajları evrendeki varlık hayattır rahmettir âlemlere rahmet gönderilmiştir rahmet'

In [ ]:
df.content=df.content.str.lower()
df.content=df.content.replace('[^a-züçöşıi̇âğî-]', " ", regex=True)
df.content=df.content.str.replace('i̇', 'i')
df.content=df.content.str.replace('-', ' ')
df.content=df.content.str.replace('   ', ' ')

df.content=df.content.str.replace("\s\w\s", " ")
df.content=df.content.str.replace("\s\w\w\s", " ")
df.content=df.content.str.replace("\sin\s", " ")
df.content=df.content.str.replace("\sın\s", " ")


In [ ]:
df.shape

(2676, 14)

In [ ]:
sw=pd.read_csv("https://raw.githubusercontent.com/InJuxSanct/turkish-stopwords/master/src/lib/stopwords/raw-stopwords.txt", sep=" ",header=None)

sw.rename(columns={0:'word'}, inplace=True)
sw=sw.word.to_list()

# df['content']=df['content'].apply(lambda x: ' '.join(x for x in x.split() if x not in sw.word.values))

In [ ]:
doc=df.content.to_list()

doc[0]

'merhamet eğitimi küçüklerimize sevgi göstermeyen değildir büyüklerimize saygı haksız yere serçeyi öldürenden cenab kıyamet gününde hesap soracaktır buyurmamış mıydı merhamet peygamberi enbiya buhari tevhid alemlere rahmet efendimize ayet nazil hayır hayır doğrusu yapıp ettiklerinden kalplerinin üzeri pas tutmuştur sahabiler kalbin pas tutmasının demek mecâzi vurgunun ifade anlamakta güçlük çekiyorlar üzerine resulü buyuruyor kul günah işlediği zaman kalbinde siyah nokta oluşur vazgeçip tövbe istiğfar zaman kalbi parlar günahtan dönmez yapma devam ederse siyah nokta arttırılır kalbini işgal eder kitabında hayır hayır'

# BerTopic

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

In [ ]:

vectorizer_model = CountVectorizer(stop_words=sw)

topic_model=BERTopic(language="Turkish", vectorizer_model=vectorizer_model, calculate_probabilities=True)

topics, probs= topic_model.fit_transform(doc)

topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1082,-1_rahmet_efendimiz_iman_kıymetli
1,0,301,0_insan_dünya_gün_vardır
2,1,225,1_namaz_okuduğum_buyuruyor_iman
3,2,150,2_islam_insan_islâm_din
4,3,121,3_rabbimizin_karşı_rabbi_eyle
5,4,73,4_aile_anne_baba_çocuklarımızı
6,5,69,5_islam_iman_muhammed_güzel
7,6,61,6_bayram_bayramıdır_bayramlar_ramazan
8,7,52,7_vatan_aynı_çanakkale_birlik
9,8,42,8_camiler_cami_görevlileri_haftası


# tunning

In [ ]:
topic_model.visualize_barchart(n_words=10, width=250, height=300, top_n_topics=33)

In [ ]:
new_topics = topic_model.reduce_outliers(doc, topics, probabilities=probs, strategy="probabilities", threshold=0.05)

topic_model.update_topics(doc, topics=new_topics)
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,508,-1_değerli_rahmet_iman_efendimiz
1,0,324,0_insan_dünya_gün_zaman
2,1,291,1_namaz_okuduğum_buyuruyor_iman
3,2,224,2_islam_karşı_islâm_insan
4,3,148,3_rabbimizin_karşı_rabbi_nun
5,4,73,4_aile_anne_baba_çocuklarımızı
6,5,130,5_islam_iman_okuduğum_buyuruyor
7,6,67,6_bayram_bayramıdır_bayramlar_ramazan
8,7,52,7_vatan_aynı_çanakkale_birlik
9,8,42,8_camiler_cami_görevlileri_haftası


In [ ]:
topic_model.visualize_barchart(n_words=10, width=250, height=300, top_n_topics=33)

In [ ]:
topic_model.visualize_documents(doc, hide_document_hover=True, hide_annotations=True)

In [ ]:
topic_model.merge_topics(doc, topics_to_merge=[6, 19, 20])
topic_model.merge_topics(doc, topics_to_merge=[2,5, 32])

In [ ]:
topic_model.visualize_barchart(n_words=10, width=250, height=300, top_n_topics=33)

In [ ]:
topic_model.visualize_documents(doc, hide_document_hover=True, hide_annotations=True)

In [ ]:
topic_model.merge_topics(doc, topics_to_merge=[9,25])
topic_model.merge_topics(doc, topics_to_merge=[10,15, 27])

In [ ]:
topic_model.visualize_barchart(n_words=10, width=250, height=300, top_n_topics=33)

In [ ]:
topic_model.visualize_documents(doc, hide_document_hover=True, hide_annotations=True)

In [ ]:
topic_model.merge_topics(doc, topics_to_merge=[0,2, 9])
topic_model.merge_topics(doc, topics_to_merge=[16,17])
topic_model.merge_topics(doc, topics_to_merge=[3,5])


In [ ]:
fig =topic_model.visualize_barchart(n_words=10, width=250, height=300, top_n_topics=33)

fig.write_image("fig1.pdf")

In [ ]:
fig2= topic_model.visualize_documents(doc, hide_document_hover=True, hide_annotations=True)

In [ ]:
topic_model.get_topic_info()

topic_labels=topic_model.generate_topic_labels(nr_words=5, topic_prefix=False, word_length=15, separator= " - ")

topic_model.set_topic_labels(topic_labels)


topic_model.set_topic_labels({-1: "Outliers", 0: "Islam (General)", 1: "Human Lifespan", 2: "Eid Celebration", 3:"Kindness and Assent", 4:"Life and Afterlife",
                              5: "Family and Friendship", 6:"Gratitude", 7:"Summer Religious School", 8:"Nation-State Discourse 1", 9:"Greeting",
                              10:'Mosques', 11: "Condolance", 12: "Holy Places", 13: 'Ethnic and Gender Focus', 14:"Holy Nights 1",
                              15: "Holy Nights 2", 16:"Nation-State Discourse 2", 17:"COVID-19 and Hygiene", 18: "Sermon Closing Remarks", 19: "Charity and Kindness",
                              20: "Heart (Feelingness)", 21:'Penitence', 22: "Honesty"})

df=topic_model.get_topic_info()

In [ ]:
topic_model.get_topic_info().head(10)

In [ ]:
print(df.to_latex(index=False))

\begin{tabular}{rrll}
\toprule
 Topic &  Count &                                          Name &               CustomName \\
\midrule
    -1 &    508 &              -1\_değerli\_rahmet\_iman\_efendimiz &                 Outliers \\
     0 &    699 &               0\_islam\_iman\_okuduğum\_buyuruyor &          Islam (General) \\
     1 &    324 &                       1\_insan\_dünya\_gün\_zaman &           Human Lifespan \\
     2 &    208 &                  2\_kurban\_bayram\_aile\_ramazan &          Eid Celebration \\
     3 &    148 &                  3\_rabbimizin\_karşı\_nun\_rabbi &      Kindness and Assent \\
     4 &    102 &                4\_rabbimizin\_edelim\_ömür\_dünya &       Life and Afterlife \\
     5 &     73 &                   5\_kardeşlik\_aile\_anne\_sevgi &    Family and Friendship \\
     6 &     68 &                  6\_rabbimizin\_şükrü\_şükür\_nun &                Gratitude \\
     7 &     62 &             7\_yaz\_kerbelâ\_ramazan\_kurslarımız &  Summer Reli

In [ ]:
import plotly.graph_objects as go
import kaleido
import plotly.io as pio


In [ ]:
fig = topic_model.visualize_barchart(n_words=10, width=250, height=300, top_n_topics=41, custom_labels=True)

# fig.layout.title="Topics "

fig.show()


In [ ]:
fig4=topic_model.visualize_barchart(n_words=10, width=250, height=300, top_n_topics=12, custom_labels=True)

fig4.show()
fig4.write_image("fig4-v1.pdf")

In [ ]:
fig5=topic_model.visualize_barchart(n_words=10, width=250, height=300, custom_labels=True, topics=[12, 13,14,15,16,17,18,19,20,21,22], title= "Topic Word Scores (Continued)")

fig5.show()
fig5.write_image("fig4-v2.pdf")

In [ ]:
fig2=topic_model.visualize_documents(doc, hide_document_hover=True, hide_annotations=True, custom_labels=True)

In [ ]:
fig2.update_traces(marker=dict(opacity= 0.8))

fig2.show()

fig2.write_image("fig2.pdf")

In [ ]:
fig6=topic_model.visualize_documents(doc, hide_document_hover=True, hide_annotations=True, custom_labels=True, topics=[8,16])
fig6.update_traces(marker=dict(opacity= 0.8))

fig6.write_image("fig6-v2.pdf")

In [ ]:
# similar_topics, similarity = topic_model.find_topics("iman", top_n=2); 

# similar_topics


[27, 29]

In [ ]:
htm=topic_model.hierarchical_topics(doc)

fig9=topic_model.visualize_hierarchy(hierarchical_topics=htm, custom_labels=True)


fig9.write_image("fig9.pdf")

fig9.show()

100%|██████████| 22/22 [00:00<00:00, 119.66it/s]


In [ ]:
topic_model.visualize_term_rank()

In [ ]:
df['date']=pd.to_datetime(df['date'],dayfirst=True)
time=df.date.to_list()
topics_over_time = topic_model.topics_over_time(doc, timestamps=time,  nr_bins=15, global_tuning=True, evolution_tuning=True)
fig8=topic_model.visualize_topics_over_time(topics_over_time, topics=[8,16], custom_labels=True)

In [ ]:
fig8.update_xaxes(
    dtick="M4",
    tickformat="%b\n%Y")
fig8.show()

fig8.write_image("fig8.pdf")

In [ ]:
topic_model.get_representative_docs(16)

['hz güven toplumu gün beytullah yanındaki safâ tepesinden mekke ahalisine seslendi vadinin arkasında size saldırmak bekleyen ordu desem inanır mısınız mekkeliler ağızdan evet inanırız emin kişisin güvenilir birisin doğruluk sadakat timsalisin yalan söylediğini işitmedik diyerek karşılık verdiler üzerine rahmet elçisi elîm azaba karşı uyarıyorum dedi mekkeliler nezdinde insanlığı imana çağırdı iman eman bulmaya türlü korku endişeden güvende olmaya davet karşılamak olağanüstü güvenlik önlemleri alınıyor teknolojinin türlü imkanı seferber bugün yerküremizi kuşatan güven bunalımı boyutlara ulaşmıştır insanlığın ortak yurdu evi',
 'tarih boyunca hak hakikat uğruna canından geçen şehitlerimizi istiklal istikbalimiz mücadele kahraman gazilerimizi hürmet rahmet şükranla yâd ediyorum hutbemi çanakkale şehitlerine ithaf edilen dizelerle bitiriyorum âsım nesli diyordum nesilmiş gerçek çiğnetmedi nâm sunu çiğnetmeyecek topraklar toprağa düşmüş asker gökten ecdâd inerek öpse pâk alnı değer büyüksü

In [ ]:

#topic_model.save("my_model")

topic_model = BERTopic.load("my_model")
